In diesem Notebook möchten wir unseren Datensatz für das trainieren der Modelle vorbereiten. Dazu müssen wir zum Beispiel einige Variablen Dummy kodieren.

In [1]:
import pandas as pd

df = pd.read_csv('DF_Join_final_bereinigt.csv')


In [2]:
import numpy as np

In [3]:
#Zunächst entfernen wir noch die Spalte imdb_id, die wir nicht ins Modell einfließen lassen

df = df.drop(['imdb_id'], 1)

<ipython-input-3-b7e5dc2987c8>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['imdb_id'], 1)


#Dummy kodieren Genres und Production Countries und Role

Leonie

In [4]:
#In der Spalte Genres wollen wir jeweils nur den ersten Eintrag behalten

df['genres'] = df['genres'].str.split(',').str[0]

In [5]:
#Für eine schönere Darstellung entfernen wir noch die Sonderzeichen

import re

def clean_Sonderzeichen(values):
    cleaned_values = []
    for x in values:
        cleaned_value = re.sub(r'[^a-zA-Z]', '', x)
        if cleaned_value:
            cleaned_values.append(cleaned_value)
    return ''.join(cleaned_values)

# Wir wenden die Funktion für die Spalte genres und production_countries an

df['genres'] = df['genres'].apply(lambda x: clean_Sonderzeichen(x))

df['production_countries'] = df['production_countries'].apply(lambda x: clean_Sonderzeichen(x))


###Zunächst Dummy kodieren wir die Spalte genres

Leonie

In [6]:
#Wir haben mehr als zwei Ausprägungen, daher nutzen wir One Hot Encoding

df.genres.value_counts()

drama            15551
comedy           10574
action            4252
thriller          4190
documentation     3044
romance           2944
crime             2404
scifi             2390
animation         1685
horror            1274
fantasy           1238
family             812
war                607
music              537
reality            318
history            215
western            208
sport               75
                     2
Name: genres, dtype: int64

In [7]:
#Wir haben zwei Spalten, die keinem Genre zugeordnet sind, daher füllen wir diese manuell auf

df[df['genres'] == '']

,name,character,role,title,description,runtime,genres,production_countries,imdb_score,imdb_votes,tmdb_score
13853,David Letterman,Self - Host,ACTOR,My Next Guest Needs No Introduction With David...,TV legend David Letterman teams up with fascin...,50.0,,US,7.8,5581.0,7.6
22977,Carol Burnett,Herself - Host,ACTOR,A Little Help with Carol Burnett,In this unscripted series starring comedy lege...,24.0,,US,6.3,237.0,6.2


In [8]:
"""Die beiden Filme werden als TV Show eingeordnet, da wir diese Ausprägung aber bisher nicht haben setzen wir beide
Filme auf die nächst passenede Ausprägung documentation"""

#Wir setzen das Genre für den ersten Film auf documentation

rows = df[df['name'] == 'David Letterman']

for index, row in rows.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

#Das Genre für den zweiten Film setzen wir auch auf documentation

rows_2 = df[df['name'] == 'Carol Burnett']

for index, row in rows_2.iterrows():
    if row['genres'] == '':
        df.loc[index, 'genres'] = 'documentation'

In [9]:
#Unser Code war erfolgreich, die 2 Ausprägungen ohne Genre sind weg und bei documentation sind 2 Werte mehr
df.genres.value_counts()

drama            15551
comedy           10574
action            4252
thriller          4190
documentation     3046
romance           2944
crime             2404
scifi             2390
animation         1685
horror            1274
fantasy           1238
family             812
war                607
music              537
reality            318
history            215
western            208
sport               75
Name: genres, dtype: int64

In [10]:
#Wir wenden nun One Hot Encoding an, löschen die genres Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='genres').join(pd.get_dummies(df.genres, drop_first = True))

###Dummy kodieren Production Countries

Leonie

In [11]:
#Wir wiederholen die Schritte für Production Countries

df.production_countries.value_counts()

US        17780
IN         4993
JP         2705
KR         2466
ES         1617
          ...  
USIL          1
EGUS          1
USCLMX        1
BEUS          1
ATUS          1
Name: production_countries, Length: 334, dtype: int64

In [12]:
#Wir haben sehr viele Kategorien, die nur einmal vorkommen um unser Modell nicht unnötig kompliziert zu machen fassen wir diese zusammen
#Damit es überschaubar bleibt setzen wir den Schwellenwert auf 100
#Die Länder kommen schließlich auch häufig vor, da die Filme mehrfach im Datensatz vorkommen, da jeder Schauspieler zu den Filmen genannt ist

threshold = 100

count = df.production_countries.value_counts()

infrequent_categories = count[count < threshold].index

df.loc[df['production_countries'].isin(infrequent_categories), 'production_countries'] = 'Other_countries'

In [13]:
df.production_countries.value_counts()

US                 17780
Other_countries     5890
IN                  4993
JP                  2705
KR                  2466
ES                  1617
GB                  1561
FR                  1068
PH                   998
ID                   935
TR                   931
CA                   870
GBUS                 722
BR                   700
DE                   697
IT                   625
USGB                 614
MX                   575
CN                   498
PL                   480
CAUS                 477
TW                   446
AU                   422
HK                   361
TH                   356
AR                   309
                     290
CO                   235
NL                   221
NG                   204
ZA                   198
USCA                 198
SE                   190
DK                   173
EG                   157
CAGBUS               148
BEFR                 147
USAU                 136
CL                   131
CNHK                 124


In [14]:
#Wir haben noch 290 Werte die keinem Land zugeordnet sind, diese setzten wir ebenfalls auf Other

df['production_countries'] = df['production_countries'].replace('', 'Other_countries')

In [15]:
df.production_countries.value_counts()

US                 17780
Other_countries     6180
IN                  4993
JP                  2705
KR                  2466
ES                  1617
GB                  1561
FR                  1068
PH                   998
ID                   935
TR                   931
CA                   870
GBUS                 722
BR                   700
DE                   697
IT                   625
USGB                 614
MX                   575
CN                   498
PL                   480
CAUS                 477
TW                   446
AU                   422
HK                   361
TH                   356
AR                   309
CO                   235
NL                   221
NG                   204
ZA                   198
USCA                 198
SE                   190
DK                   173
EG                   157
CAGBUS               148
BEFR                 147
USAU                 136
CL                   131
CNHK                 124
DEUS                 117


In [16]:
#Wir wenden nun One Hot Encoding an, löschen die production countries Spalte und hängen die neuen an unseren Datensatz an

df = df.drop(columns='production_countries').join(pd.get_dummies(df.production_countries, drop_first = True))
df.head(5)

,name,character,role,title,description,runtime,imdb_score,imdb_votes,tmdb_score,animation,...,SE,TH,TR,TW,US,USAU,USCA,USGB,XX,ZA
0,Graham Chapman,King Arthur / Voice of God / Middle Head / Hic...,ACTOR,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,...,0,0,0,0,0,0,0,0,0,0
1,John Cleese,Second Swallow-Savvy Guard / The Black Knight ...,ACTOR,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,...,0,0,0,0,0,0,0,0,0,0
2,Eric Idle,Dead Collector / Peasant 1 / Sir Robin the Not...,ACTOR,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,...,0,0,0,0,0,0,0,0,0,0
3,Terry Gilliam,Patsy / Green Knight / Old Man from Scene 24 (...,ACTOR,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,...,0,0,0,0,0,0,0,0,0,0
4,Terry Jones,Dennis's Mother / Sir Bedevere / Left Head / P...,ACTOR,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,...,0,0,0,0,0,0,0,0,0,0


##Dummy Kodierung der Spalte Rolle

In [17]:
#Wir schauen zunächst wieder welche Werte die Spalte role hat

df.role.value_counts()

ACTOR    52320
Name: role, dtype: int64

In [18]:
#Wir haben nur Schauspieler im Datensatz, daher können wir diese Spalte einfach entfernen
df = df.drop(['role'], 1)

<ipython-input-18-9725e351777d>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(['role'], 1)


# Dummy kodieren Schauspieler & Top 100 Schauspieler

Madeleine

In [19]:
#Zunächste formatieren wir die Namen der Schauspieler um, um später die Top-100 Schauspielernamen richtig formatiert als Spaltennamen zu haben
df['name'] = df['name'].str.replace(' ', '_')
df

,name,character,title,description,runtime,imdb_score,imdb_votes,tmdb_score,animation,comedy,...,SE,TH,TR,TW,US,USAU,USCA,USGB,XX,ZA
0,Graham_Chapman,King Arthur / Voice of God / Middle Head / Hic...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,...,0,0,0,0,0,0,0,0,0,0
1,John_Cleese,Second Swallow-Savvy Guard / The Black Knight ...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Eric_Idle,Dead Collector / Peasant 1 / Sir Robin the Not...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Terry_Gilliam,Patsy / Green Knight / Old Man from Scene 24 (...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Terry_Jones,Dennis's Mother / Sir Bedevere / Left Head / P...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52315,Kalia,Kalia,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,7.3,36.0,6.2,1,0,...,0,0,0,0,0,0,0,0,0,0
52316,Dholu,Dholu,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,7.3,36.0,6.2,1,0,...,0,0,0,0,0,0,0,0,0,0
52317,Bholu,Bholu,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,7.3,36.0,6.2,1,0,...,0,0,0,0,0,0,0,0,0,0
52318,Raju,Raju,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,7.3,36.0,6.2,1,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#Wir wollen einmal zählen, wie oft jeder Schauspieler im Datensatz vorkommt, um die Top-10 Schauspieler ausfindig zu machen

count= df.name.value_counts()
count

Shah_Rukh_Khan           23
Boman_Irani              22
Kareena_Kapoor_Khan      19
Aamir_Khan               18
Priyanka_Chopra_Jonas    17
                         ..
Julian_Barratt            1
Pasqualina_Cardu          1
Isabel_Dove               1
Neil_Whitely              1
Raju                      1
Name: name, Length: 38725, dtype: int64

In [21]:
#hier lassen wir uns einmal die Verteilung anzeigen, wie die Anzahl der Schauspieler im Datensatz verteilt ist
#damit können wir definieren, wo wir unsere Grenze der Top-Schauspieler definieren möchten
frequency_count = count.value_counts().sort_index()
print(frequency_count)

1     31161
2      4706
3      1572
4       608
5       293
6       148
7        68
8        44
9        33
10       29
11       21
12       17
13        5
14        3
15        4
16        6
17        3
18        1
19        1
22        1
23        1
Name: name, dtype: int64


In [22]:
#wir werden für unsere weiteren Analysen lediglich die Top-100 Schauspieler betrachten
top_100_names = count.head(100)
print(top_100_names)

Shah_Rukh_Khan           23
Boman_Irani              22
Kareena_Kapoor_Khan      19
Aamir_Khan               18
Priyanka_Chopra_Jonas    17
                         ..
Cho                       9
Ashleigh_Ball             9
Adil_Hussain              9
Tomokazu_Sugita           9
Ricky_Gervais             9
Name: name, Length: 100, dtype: int64


In [23]:
#hierfür müssen wir zunächst den Nicht-Top-100-Schauspielern den Wert 'Other'zuweisen für die spätere Dummybildung
top_100_names = df['name'].value_counts().head(100).index
df['name_encoded'] = np.where(df['name'].isin(top_100_names), df['name'], 'Other_actors')
#wir testen, ob die Zuordnung erfolgreich war am Beispiel für einen unserer Top 100 Schauspieler
test=df.loc[df['name']=='Cho']
test

,name,character,title,description,runtime,imdb_score,imdb_votes,tmdb_score,animation,comedy,...,TH,TR,TW,US,USAU,USCA,USGB,XX,ZA,name_encoded
635,Cho,Brook (voice),One Piece,"Years ago, the fearsome Pirate King, Gol D. Ro...",23.0,8.8,117129.0,8.8,1,0,...,0,0,0,0,0,0,0,0,0,Cho
2623,Cho,Brook (voice),One Piece: Strong World,"20 years after his escape from Impel Down, the...",114.0,7.5,5467.0,7.5,0,0,...,0,0,0,0,0,0,0,0,0,Cho
3749,Cho,Jaken,Inuyasha the Movie: Affections Touching Across...,InuYasha is a half-demon who was trapped in th...,99.0,7.2,2593.0,7.6,0,0,...,0,0,0,0,0,0,0,0,0,Cho
3773,Cho,Jaken,Inuyasha the Movie 4: Fire on the Mystic Island,The mysterious island of Houraijima has reappe...,88.0,7.2,1845.0,7.7,0,0,...,0,0,0,0,0,0,0,0,0,Cho
3909,Cho,Jaken,Inuyasha the Movie 3: Swords of an Honorable R...,"Inuyasha and his brother, Sesshomaru, each inh...",99.0,7.6,2226.0,7.8,1,0,...,0,0,0,0,0,0,0,0,0,Cho
5429,Cho,Brook (voice),One Piece Film: Z,"Zephyr, now known as Z, rides the seas with on...",108.0,7.8,7243.0,7.5,0,0,...,0,0,0,0,0,0,0,0,0,Cho
21899,Cho,Jiiya (voice),Hi Score Girl,The year is 1991 and 6th grader Yaguchi Haruo ...,27.0,8.0,2002.0,8.2,0,1,...,0,0,0,0,0,0,0,0,0,Cho
22457,Cho,Ebina's Father (voice),Lu Over the Wall,"In an small fishing village, a gloomy middle s...",117.0,6.8,2269.0,7.0,1,0,...,0,0,0,0,0,0,0,0,0,Cho
36449,Cho,Kazuo Yamashita (voice),Kengan Ashura,A timid old man is summoned by his chairman to...,25.0,8.0,4869.0,8.3,1,0,...,0,0,0,0,0,0,0,0,0,Cho


In [24]:
#Hier bilden wir für die Top 100 Schauspieler die Dummies und löschen die Zeilen 'name' und 'name_encoded'
dummies = pd.get_dummies(df['name_encoded'], drop_first=True)
df = pd.concat([df, dummies], axis=1)
df = df.drop(columns=['name', 'name_encoded'])
df.head(5)


,character,title,description,runtime,imdb_score,imdb_votes,tmdb_score,animation,comedy,crime,...,Tara_Strong,Terry_Gilliam,Terry_Jones,Tiffany_Haddish,Tomokazu_Sugita,Vijay_Raaz,Vincent_Tong,Vir_Das,Yoshimasa_Hosoya,Yuki_Kaji
0,King Arthur / Voice of God / Middle Head / Hic...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Second Swallow-Savvy Guard / The Black Knight ...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dead Collector / Peasant 1 / Sir Robin the Not...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Patsy / Green Knight / Old Man from Scene 24 (...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,Dennis's Mother / Sir Bedevere / Left Head / P...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,8.2,534486.0,7.8,0,0,0,...,0,0,1,0,0,0,0,0,0,0


#Zielvariable berechnen
Leonie

In [25]:
'''Unsere Zielvariable für die Regression ist der imdb und tmdb Score, diese möchten wir in einem gewichteten Mittelwert
(75% imdb score 25% tmdb score) kodieren'''

weight_imdb = 0.75
weight_tmdb = 0.25

df['target_variable'] = ((df['imdb_score'] * weight_imdb) + (df['tmdb_score'] * weight_tmdb))

In [26]:
#Die ursprünglichen Spalten löschen wir

df = df.drop(columns=['imdb_score', 'tmdb_score'])

In [27]:
df.isna().any()

character           False
title               False
description         False
runtime             False
imdb_votes          False
                    ...  
Vincent_Tong        False
Vir_Das             False
Yoshimasa_Hosoya    False
Yuki_Kaji           False
target_variable     False
Length: 167, dtype: bool

In [28]:
df

,character,title,description,runtime,imdb_votes,animation,comedy,crime,documentation,drama,...,Terry_Gilliam,Terry_Jones,Tiffany_Haddish,Tomokazu_Sugita,Vijay_Raaz,Vincent_Tong,Vir_Das,Yoshimasa_Hosoya,Yuki_Kaji,target_variable
0,King Arthur / Voice of God / Middle Head / Hic...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,534486.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.100
1,Second Swallow-Savvy Guard / The Black Knight ...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,534486.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.100
2,Dead Collector / Peasant 1 / Sir Robin the Not...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,534486.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.100
3,Patsy / Green Knight / Old Man from Scene 24 (...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,534486.0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8.100
4,Dennis's Mother / Sir Bedevere / Left Head / P...,Monty Python and the Holy Grail,"King Arthur, accompanied by his squire, recrui...",91.0,534486.0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,8.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52315,Kalia,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,36.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.025
52316,Dholu,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,36.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.025
52317,Bholu,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,36.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.025
52318,Raju,Chhota Bheem Aur Hanuman,"The grouchy Makhichoos, whoe hates Diwali, is ...",67.0,36.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7.025


#Speichern der Daten

In [29]:
df.to_csv("Daten_kodiert_fuer_modelle.csv", index=False)